In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import os
import librosa
import librosa.display
import IPython.display as ipd
from itertools import cycle
from tqdm import tqdm

In [3]:
class Configuration:

  class Paths:
    train_audio_folders = '/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_audio'
    train_audio_files='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_audio/*/*.ogg'
    unlabeld_soundspaces='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/unlabeled_soundscapes'
    taxonomy_birds = '/content/drive/MyDrive/Kaggle/Bird_CLEF2024/eBird_Taxonomy_v2021.csv'
    meta_data='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_metadata.csv'
    sample_submission='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/sample_submission.csv'


  class ClassInfo:
    class_names=sorted(os.listdir('/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_audio'))
    num_classes=len(class_names)
    labels=list(range(num_classes))
    labels_with_names = dict(zip(labels,class_names))

config=Configuration()

In [4]:
labels_names=config.ClassInfo.labels_with_names
meta_data=pd.read_csv(config.Paths.meta_data)
meta_data['filepath'] = config.Paths.train_audio_folders+'/'+meta_data.filename

In [5]:
meta_data.shape

(24459, 13)

In [6]:
# Function to load and process audio in segments
def process_audio_segmented(file_paths, segment_length=10):
    audio_data = []
    for file_path in tqdm(file_paths, desc="Processing Audio"):
        y, sr = librosa.load(file_path, sr=None, mono=True)
        num_segments = int(np.ceil(len(y) / (sr * segment_length)))
        for i in range(num_segments):
            start_sample = i * sr * segment_length
            end_sample = min((i + 1) * sr * segment_length, len(y))
            segment = y[start_sample:end_sample]
            audio_data.append(segment)
    return audio_data

# Your list of file paths
file_paths = meta_data['filepath'].tolist()

# Process audio files in segments
audio_data = process_audio_segmented(file_paths)

NameError: name 'tqdm' is not defined

In [ ]:
import librosa
from tqdm import tqdm
import numpy as np
import concurrent.futures

# Define a function to load audio files using memory mapping
def load_audio_mm(file_path):
    y, sr = librosa.load(file_path, sr=None, mono=True)
    return y, sr

# Process audio files using memory mapping
def process_audio(files):
    audio_data = []
    with concurrent.futures.ProcessPoolExecutor() as executor:
        results = list(tqdm(executor.map(load_audio_mm, files), total=len(files)))
    for result in results:
        audio_data.append(result)
    return audio_data

# Your list of file paths
file_paths = meta_data['filepath'].tolist()

# Process audio files
audio_data = process_audio(file_paths)
